In [1]:
'''import tweepy

client = tweepy.Client(bearer_token='AAAAAAAAAAAAAAAAAAAAAOrPYAEAAAAAzotm%2BluowpF8blMB3kwWGXk%2B%2BJU%3DrKoUdWu5nquE4NaYEQ27TJVo103unUVbW5JQmY8L4GIZckjHwp') 

# Replace with your own search query
query = 'vaccine -is:retweet place_country:US'

tweets = client.search_all_tweets(query=query, tweet_fields=['context_annotations', 'created_at', 'geo'],
                                  place_fields=['place_type', 'geo'], expansions='geo.place_id', max_results=100)

# Get list of places from includes object
places = {p["id"]: p for p in tweets.includes['places']}

for tweet in tweets.data:
    print(tweet.id)
    print(tweet.text)
    if places[tweet.geo['place_id']]:
        place = places[tweet.geo['place_id']]
        print(place.full_name)
'''

'import tweepy\n\nclient = tweepy.Client(bearer_token=\'AAAAAAAAAAAAAAAAAAAAAOrPYAEAAAAAzotm%2BluowpF8blMB3kwWGXk%2B%2BJU%3DrKoUdWu5nquE4NaYEQ27TJVo103unUVbW5JQmY8L4GIZckjHwp\') \n\n# Replace with your own search query\nquery = \'vaccine -is:retweet place_country:US\'\n\ntweets = client.search_all_tweets(query=query, tweet_fields=[\'context_annotations\', \'created_at\', \'geo\'],\n                                  place_fields=[\'place_type\', \'geo\'], expansions=\'geo.place_id\', max_results=100)\n\n# Get list of places from includes object\nplaces = {p["id"]: p for p in tweets.includes[\'places\']}\n\nfor tweet in tweets.data:\n    print(tweet.id)\n    print(tweet.text)\n    if places[tweet.geo[\'place_id\']]:\n        place = places[tweet.geo[\'place_id\']]\n        print(place.full_name)\n'

In [2]:

# For sending GET requests from the API
import requests
# For saving access tokens and for file management when creating and adding to the dataset
import os
# For dealing with json responses we receive from the API
import json
# For displaying the data after
import pandas as pd
# For saving the response data in CSV format
import csv
# For parsing the dates received from twitter in readable formats
import datetime
import dateutil.parser
import unicodedata
#To add wait time between requests
import time

In [3]:
os.environ['TOKEN'] = 'AAAAAAAAAAAAAAAAAAAAAOrPYAEAAAAAzotm%2BluowpF8blMB3kwWGXk%2B%2BJU%3DrKoUdWu5nquE4NaYEQ27TJVo103unUVbW5JQmY8L4GIZckjHwp'


In [4]:

def auth():
    return os.getenv('TOKEN')

In [5]:

def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers

In [6]:
def create_url(keyword, start_date, end_date, max_results = 10):
    
    search_url = "https://api.twitter.com/2/tweets/search/all" #Change to the endpoint you want to collect data from

    #change params based on the endpoint you are using
    query_params = {'query': keyword,
                    'start_time': start_date,
                    'end_time': end_date,
                    'max_results': max_results,  
                    'expansions': 'author_id,in_reply_to_user_id,geo.place_id',
                    'tweet.fields': 'id,text,author_id,in_reply_to_user_id,geo,conversation_id,created_at,lang,public_metrics,referenced_tweets,reply_settings,source',
                    'user.fields': 'id,name,username,created_at,description,public_metrics,verified',
                    'place.fields': 'full_name,id,country,country_code,geo,name,place_type',
                    'next_token': {}}
    return (search_url, query_params)

In [7]:

def connect_to_endpoint(url, headers, params, next_token = None):
    params['next_token'] = next_token   #params object received from create_url function
    response = requests.request("GET", url, headers = headers, params = params)
    #print("Endpoint Response Code: " + str(response.status_code))
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

In [8]:
'''
#Inputs for the request
bearer_token = auth()
headers = create_headers(bearer_token)
keyword = "vaccine lang:en place_country:US"
start_time = "2021-03-01T00:00:00.000Z"
end_time = "2021-03-31T00:00:00.000Z"
max_results = 15
'''

'\n#Inputs for the request\nbearer_token = auth()\nheaders = create_headers(bearer_token)\nkeyword = "vaccine lang:en place_country:US"\nstart_time = "2021-03-01T00:00:00.000Z"\nend_time = "2021-03-31T00:00:00.000Z"\nmax_results = 15\n'

In [9]:
'''
url = create_url(keyword, start_time,end_time, max_results)
json_response = connect_to_endpoint(url[0], headers, url[1])
'''

'\nurl = create_url(keyword, start_time,end_time, max_results)\njson_response = connect_to_endpoint(url[0], headers, url[1])\n'

In [10]:
#print(json.dumps(json_response, indent=4, sort_keys=True))


In [11]:
#json_response['data'][0]['created_at']


In [12]:
#json_response


In [13]:
'''
with open('data.json', 'w') as f:
    json.dump(json_response, f)
'''

"\nwith open('data.json', 'w') as f:\n    json.dump(json_response, f)\n"

In [14]:

# Create file
csvFile = open("data.csv", "a", newline="", encoding='utf-8')
csvWriter = csv.writer(csvFile)

#Create headers for the data you want to save, in this example, we only want save these columns in our dataset
csvWriter.writerow(['author id', 'created_at', 'geo','bbox', 'id','lang', 'like_count', 'quote_count', 'reply_count','retweet_count','source','tweet'])
csvFile.close()

In [15]:
def append_to_csv(json_response, fileName):

    #A counter variable
    counter = 0

    #Open OR create the target CSV file
    csvFile = open(fileName, "a", newline="", encoding='utf-8')
    csvWriter = csv.writer(csvFile)
    
    # Get list of places from includes object
    places = {p["id"]: p for p in json_response["includes"]['places']}

    #Loop through each tweet
    for tweet in json_response['data']:
        
        # We will create a variable for each since some of the keys might not exist for some tweets
        # So we will account for that

        
        try:
            places[tweet["geo"]['place_id']]
            place = places[tweet["geo"]['place_id']]
            geo =place["full_name"]
            bbox =place["geo"]["bbox"]
        except:
            continue

            
        # 1. Author ID
        author_id = tweet['author_id']

        # 2. Time created
        created_at = dateutil.parser.parse(tweet['created_at'])

        # 3. Geolocation
        #geo = tweet["geo"]['place_id']

        # 4. Tweet ID
        tweet_id = tweet['id']

        # 5. Language
        lang = tweet['lang']

        # 6. Tweet metrics
        retweet_count = tweet['public_metrics']['retweet_count']
        reply_count = tweet['public_metrics']['reply_count']
        like_count = tweet['public_metrics']['like_count']
        quote_count = tweet['public_metrics']['quote_count']

        # 7. source
        source = tweet['source']

        # 8. Tweet text
        text = tweet['text']
        
        # Assemble all data in a list
        res = [author_id, created_at, geo, bbox, tweet_id, lang, like_count, quote_count, reply_count, retweet_count, source, text]
        
        # Append the result to the CSV file
        csvWriter.writerow(res)
        counter += 1

    # When done, close the CSV file
    csvFile.close()

    # Print the number of tweets for this iteration
    #print("# of Tweets added from this response: ", counter) 

In [16]:

# Create file
csvFile = open("data.csv", "a", newline="", encoding='utf-8')
csvWriter = csv.writer(csvFile)

#Create headers for the data you want to save, in this example, we only want save these columns in our dataset
csvWriter.writerow(['author id', 'created_at', 'geo', 'bbox', 'id','lang', 'like_count', 'quote_count', 'reply_count','retweet_count','source','tweet'])
csvFile.close()


In [17]:
#Inputs for tweets
bearer_token = auth()
headers = create_headers(bearer_token)
keyword = "vaccines lang:en place_country:US OR vaccine lang:en place_country:US OR pfizer lang:en place_country:US OR moderna lang:en place_country:US OR vaccinated lang:en place_country:US OR vaccination lang:en place_country:US"   

#keyword = "xbox lang:en"

start_list =    ['2021-01-01T00:00:00.000Z']
end_list =      ['2021-07-31T00:00:00.000Z']
max_results = 500

#Total number of tweets we collected from the loop
total_tweets = 0

for i in range(0,len(start_list)):

    # Inputs
    count = 0 # Counting tweets per time period
    max_count = 3000000 # Max tweets per time period
    flag = True
    next_token = None
    
    # Check if flag is true
    while flag:
        # Check if max_count reached
        if count >= max_count:
            break

        url = create_url(keyword, start_list[i],end_list[i], max_results)
        json_response = connect_to_endpoint(url[0], headers, url[1], next_token)
        result_count = json_response['meta']['result_count']

        if 'next_token' in json_response['meta']:
            # Save the token to use for next call
            next_token = json_response['meta']['next_token']
            if result_count is not None and result_count > 0 and next_token is not None:
                append_to_csv(json_response, "data.csv")
                count += result_count
                total_tweets += result_count
                print("Total # of Tweets added: ", total_tweets)
                time.sleep(5)                
        # If no next token exists
        else:
            if result_count is not None and result_count > 0:

                append_to_csv(json_response, "data.csv")
                count += result_count
                total_tweets += result_count
                print("Total # of Tweets added: ", total_tweets)
                time.sleep(5)
            
            #Since this is the final request, turn flag to false to move to the next time period.
            flag = False
            next_token = None
        time.sleep(5)
print("Total number of results: ", total_tweets)

Total # of Tweets added:  498
Total # of Tweets added:  998
Total # of Tweets added:  1497
Total # of Tweets added:  1997
Total # of Tweets added:  2494
Total # of Tweets added:  2993
Total # of Tweets added:  3492
Total # of Tweets added:  3990
Total # of Tweets added:  4489
Total # of Tweets added:  4987
Total # of Tweets added:  5484
Total # of Tweets added:  5983
Total # of Tweets added:  6482
Total # of Tweets added:  6979
Total # of Tweets added:  7477
Total # of Tweets added:  7975
Total # of Tweets added:  8475
Total # of Tweets added:  8971
Total # of Tweets added:  9465
Total # of Tweets added:  9965
Total # of Tweets added:  10465
Total # of Tweets added:  10964
Total # of Tweets added:  11462
Total # of Tweets added:  11960
Total # of Tweets added:  12459
Total # of Tweets added:  12958
Total # of Tweets added:  13457
Total # of Tweets added:  13955
Total # of Tweets added:  14454
Total # of Tweets added:  14951
Total # of Tweets added:  15450
Total # of Tweets added:  1594

Total # of Tweets added:  127340
Total # of Tweets added:  127833
Total # of Tweets added:  128326
Total # of Tweets added:  128814
Total # of Tweets added:  129312
Total # of Tweets added:  129807
Total # of Tweets added:  130302
Total # of Tweets added:  130792
Total # of Tweets added:  131281
Total # of Tweets added:  131775
Total # of Tweets added:  132270
Total # of Tweets added:  132759
Total # of Tweets added:  133250
Total # of Tweets added:  133743
Total # of Tweets added:  134237
Total # of Tweets added:  134732
Total # of Tweets added:  135228
Total # of Tweets added:  135722
Total # of Tweets added:  136216
Total # of Tweets added:  136713
Total # of Tweets added:  137209
Total # of Tweets added:  137705
Total # of Tweets added:  138198
Total # of Tweets added:  138695
Total # of Tweets added:  139193
Total # of Tweets added:  139687
Total # of Tweets added:  140179
Total # of Tweets added:  140668
Total # of Tweets added:  141162
Total # of Tweets added:  141645
Total # of

Total # of Tweets added:  250254
Total # of Tweets added:  250752
Total # of Tweets added:  251249
Total # of Tweets added:  251744
Total # of Tweets added:  252235
Total # of Tweets added:  252731
Total # of Tweets added:  253226
Total # of Tweets added:  253725
Total # of Tweets added:  254223
Total # of Tweets added:  254709
Total # of Tweets added:  255203
Total # of Tweets added:  255698
Total # of Tweets added:  256191
Total # of Tweets added:  256682
Total # of Tweets added:  257174
Total # of Tweets added:  257668
Total # of Tweets added:  258160
Total # of Tweets added:  258657
Total # of Tweets added:  259154
Total # of Tweets added:  259648
Total # of Tweets added:  260143
Total # of Tweets added:  260641
Total # of Tweets added:  261134
Total # of Tweets added:  261621
Total # of Tweets added:  262117
Total # of Tweets added:  262612
Total # of Tweets added:  263109
Total # of Tweets added:  263604
Total # of Tweets added:  264099
Total # of Tweets added:  264595
Total # of

PermissionError: [Errno 13] Permission denied: 'data.csv'